# Visualization of Data from Formspring 

Involves "smart" parsing, specifically: convert words into sounds and then back into a dictionary to see if we can decipher some of the slang used in online communication. 


In [1]:
%load_ext autoreload
%autoreload 2

In [3]:
import math
import random
import sys
import os
from util.Util import *
from util.parser import formspring_data_parser
import pandas as pd
from sklearn import model_selection

In [4]:
import sys
sys.path.append("/Users/luisd/dev/cyberbullying-detection")
# from nn import *
# import nn 
# import fc_nn 

Using TensorFlow backend.


In [ ]:
# sys.path.append(...) # in case we want to add something else

In [ ]:
import phonemes_from_graphemes as pg
from words_2_vectors import * 
from util import formspring_data_parser

In [17]:
# import importlib
# importlib.reload(pg)
from phonemes_from_graphemes import *

### Where am I expecting to see the data

In [5]:
data_dir = "./data"

In [6]:
common_logger = get_logger(name = "common_logger", debug_log_file_name = "common_logger.log")

2017-06-14 14:40:30,665 - common_logger - INFO - 'common_logger': logging 'INFO'+ logs to Console, 'DEBUG'+ logs to '/Users/luisd/dev/cyberbullying-detection/common_logger.log'


Logger created
Creating debug handler at '/Users/luisd/dev/cyberbullying-detection/common_logger.log'
'common_logger': logging 'INFO'+ logs to Console, 'DEBUG'+ logs to '/Users/luisd/dev/cyberbullying-detection/common_logger.log'


In [7]:
common_logger.handlers[1].baseFilename

'/Users/luisd/dev/cyberbullying-detection/common_logger.log'

In [28]:
from util.parser.formspring_data_parser import * 
from util.parser.wikipedia_talk_parser import *

In [10]:
a_formspring_data_parser = Formspring_Data_Parser(all_data = pd.read_csv("really_all_of_them.csv"), alogger = common_logger)

## Load Word2Vec model trained with Google News 

In [13]:
mw = ModelWrapper.from_google_news_model(data_dir=data_dir, alogger=common_logger)

2017-06-14 14:45:46,093 - common_logger - INFO - Loading model from ./data/GoogleNews-vectors-negative300.bin.gz...
2017-06-14 14:48:11,008 - common_logger - INFO - Model succesfully loaded
2017-06-14 14:48:11,010 - common_logger - INFO - Sort all the words in the model, so that we can auto-complete queries quickly...


In [14]:
m = mw.model # cache model definition

In [15]:
len(m.vocab)

3000000

## Creation of Sounds' Dictionary  

### Let's read the sounds' dictionary
(and minimally sanit-check it) 

In [18]:
sounds_dict = SoundsDict(a_dir=data_dir, alogger=common_logger)

In [19]:
sounds_dict['ju:']

{'YOU', 'YOu', 'You', 'you'}

In [20]:
mw.set_sounds_dict(sounds_dict=sounds_dict)

In [21]:
mw.sound_to_word('ju:')

{'YOU', 'YOu', 'You', 'you'}

## Read data from XML  

In [22]:
xml_file_name = '/Users/luisd/Downloads/FormspringLabeledForCyberbullying/XMLMergedFile.xml'
phonemesFactory = PhonemesFromGraphemes(alogger=common_logger)


TypeError: 'list' object cannot be interpreted as an integer

In [24]:
with open(xml_file_name) as fd:
    the_doc = xmltodict.parse(fd.read())
how_many_entries(the_doc)

50

In [25]:
type(the_doc)

collections.OrderedDict

In [ ]:
wiki_parser = Wikipedia_Talk_Parser.from_sources(pg = phonemesFactory, mw = mw, alogger=common_logger)

2017-06-14 16:47:08,058 - common_logger - INFO - 'https://ndownloader.figshare.com/files/7554634' is downloaded as 'attack_annotated_comments.tsv'
2017-06-14 16:47:13,801 - common_logger - INFO - 'https://ndownloader.figshare.com/files/7554637' is downloaded as 'attack_annotations.tsv'


In [ ]:
1 + 1

In [ ]:
wiki_parser.save("dataframe_wiki.csv")

In [26]:
parser = Formspring_Data_Parser.from_raw_xml(xml_file_name, pg = phonemesFactory, mw = mw, alogger = common_logger)

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49]


TypeError: __init__() missing 1 required positional argument: 'self'

In [ ]:
# really_all = parser.all_questions_answers_labels()

In [ ]:
# really_all.to_csv("really_all_of_them.csv")

In [ ]:
really_all = pd.read_csv("really_all_of_them.csv")

In [ ]:
threatening_items = really_all.loc[really_all["threat"] == True]

In [ ]:
print("threatening_items.shape = {}".format(threatening_items.shape))
print("really_all.shape = {}".format(really_all.shape))
print("There are {}/{} interactions ({:.2f}% of total) that are threatening".format(threatening_items.shape[0], really_all.shape[0], threatening_items.shape[0] * 100/really_all.shape[0]))

In [ ]:
questions_answers = ["{}; {}".format(q, a) for q, a in list(zip(really_all['question'].tolist(), really_all['answer'].tolist()))] 

In [ ]:
questions_answers[0:10] 

In [ ]:
labels = ["THREAT" if threat else "CLEAN" for threat in really_all['threat'].tolist()]

In [ ]:
labels[0:10]

In [ ]:
from collections import Counter
import numpy as np

In [ ]:
# Create three Counter objects to store positive, negative and total counts
positive_counts = Counter()
negative_counts = Counter()
total_counts = Counter()
for label, review in zip(labels, questions_answers):
    words = review.split(' ') 
    if label == 'THREAT':
        for word in words:
            negative_counts[word] += 1
    else:
        for word in words:
            positive_counts[word] += 1
    for word in words:
        total_counts[word] += 1

pos_neg_ratios = Counter()

unique_words = total_counts.keys()
for word in unique_words:
    if total_counts[word] >= 100:
        pos_neg_ratios[word] = positive_counts[word] / float(negative_counts[word]+1)
        
        
unique_words = pos_neg_ratios.keys()# set(pos_neg_ratios.elements())
for word in unique_words:
    if (pos_neg_ratios[word] > 1):
        pos_neg_ratios[word] = np.log(pos_neg_ratios[word])
    else:
        pos_neg_ratios[word] = -np.log(1/(pos_neg_ratios[word] + 0.01))        

In [ ]:
negative_counts.most_common()

In [ ]:
# negative_counts.most_common()
print("Pos-to-neg ratio for 'the' = {}".format(pos_neg_ratios["the"]))
print("Pos-to-neg ratio for 'amazing' = {}".format(pos_neg_ratios["amazing"]))
print("Pos-to-neg ratio for 'shit' = {}".format(pos_neg_ratios["shit"]))

### What are the most common words for 'threats' and 'non-threats' 

In [ ]:
pos_neg_ratios.most_common()

In [ ]:
list(reversed(pos_neg_ratios.most_common()))[0:30]

### Yeah... doesn't look like there is a pattern here... 
Maybe some more work on the pre-treatment is needed 

### Let's take a look 

In [ ]:
from bokeh.models import ColumnDataSource, LabelSet
from bokeh.plotting import figure, show, output_file
from bokeh.io import output_notebook
output_notebook()

In [ ]:
hist, edges = np.histogram(list(map(lambda x:x[1],pos_neg_ratios.most_common())), density=True, bins=100, normed=True)

p = figure(tools="pan,wheel_zoom,reset,save",
           toolbar_location="above",
           title="Word Positive/Negative Affinity Distribution")
p.quad(top=hist, bottom=0, left=edges[:-1], right=edges[1:], line_color="#555555")
show(p)

In [ ]:
frequency_frequency = Counter()

for word, cnt in total_counts.most_common():
    frequency_frequency[cnt] += 1

In [ ]:
hist, edges = np.histogram(list(map(lambda x:x[1],frequency_frequency.most_common())), density=True, bins=100, normed=True)

p = figure(tools="pan,wheel_zoom,reset,save",
           toolbar_location="above",
           title="The frequency distribution of the words in our corpus")
p.quad(top=hist, bottom=0, left=edges[:-1], right=edges[1:], line_color="#555555")
show(p)

In [ ]:
vocab = set(total_counts.keys())
vocab_size = len(vocab)
print(vocab_size)